## Created by <a href="https://github.com/yunsuxiaozi">yunsuxiaozi</a> 2024/6/21

#### 这是优酷x天池 「酷文」小说创作大模型挑战赛的baseline.我这里使用的是谷歌的Gemma模型(pytorch-2b-it-V2)。使用这个模型作为baseline没有什么理由,就是因为我是大模型的初学者,对这块了解的不多,目前所掌握的大模型调用只会这一个,Gemma模型刚出的时候感兴趣就学了一下,仅此而已。后续我也会学习NLP包括大模型的相关知识。

#### 导入库,并读取测试数据,这个测试数据和普通的json文件读取方式不同,后面保存提交文件也和普通的json文件保存方式不同。

In [1]:
import pandas as pd#导入csv文件的库
import json#用于读取和写入json数据格式
test=[]
with open("/kaggle/input/llmnovel/write novels  LLM/test.json",'r',encoding='utf-8') as f:
    for line in f:
        test.append(json.loads(line))

#### 查看数据

In [2]:
Instruction=[test[i]['instruction'] for i in range(len(test))]
Input=[test[i]['input'] for i in range(len(test))]
Output=[test[i]['output'] for i in range(len(test))]
print(f"sample_Instruction:{Instruction[0]},sample_input:{Input[0]},sample_output:{Output[0]}")

sample_Instruction:,sample_input:现代励志故事，一个失业青年如何克服生活困境，终于实现自我突破，成为行业翘楚的心路历程。,sample_output:


#### Gemma

In [3]:
# 尝试使用gemma https://www.kaggle.com/code/shresthshukla/gemma-using-pytorch
#安装 -q(减少输出信息) -U是升级,不可变的字典 Byte Pair Encoding, BPE的分词库
!pip install -q -U immutabledict sentencepiece
#克隆github 的gemma_pytorch仓库
!git clone https://github.com/google/gemma_pytorch.git
#将克隆来的全部东西移动到本地
!mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/

Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 177 (delta 64), reused 68 (delta 46), pack-reused 68
Receiving objects: 100% (177/177), 2.16 MiB | 23.32 MiB/s, done.
Resolving deltas: 100% (91/91), done.
mv: target '/kaggle/working/gemma/' is not a directory


In [4]:
import sys#允许python开发者与python解释器进行交互
#如果你尝试导入一个模块,如果模块不在其他目录下,还会在新添加的目录进行查找
sys.path.append("/kaggle/working/gemma_pytorch/") 
#获取7b模型的参数和获取2b模型的参数
from gemma.config import get_config_for_7b, get_config_for_2b
#Causal Language Model，简称 CLM,因果语言模型
from gemma.model import GemmaForCausalLM
#导入gemma的分词器
from gemma.tokenizer import Tokenizer
import contextlib#简化上下文管理器
#pytorch
import torch

In [5]:
import random#提供了一些用于生成随机数的函数
VARIANT = "2b-it" #选择模型的名字
weights_dir = '/kaggle/input/gemma/pytorch/2b-it/2' #模型参数的路径
device='cuda' if torch.cuda.is_available() else"cpu"
ckpt_path = f'{weights_dir}/gemma-{VARIANT}.ckpt'#checkpoint,深度学习模型在训练过程中的状态
def seed_everything(seed):
    torch.backends.cudnn.deterministic = True#将cuda加速的随机数生成器设为确定性模式
    torch.backends.cudnn.benchmark = True#关闭CuDNN框架的自动寻找最优卷积算法的功能，以避免不同的算法对结果产生影响
    torch.manual_seed(seed)#pytorch的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=2024)

In [6]:
"""
kimi chat
bfloat16 半精度浮点数 8位指数7位尾数(小数部分) 数据范围大,精度低
float16是5位指数,10位尾数 数据范围小,精度高
硬件兼容性：并非所有的硬件都支持 bfloat16。在某些情况下，尤其是在推理时，硬件可能只支持标准的 float32 格式。因此，为了确保模型能够在这些硬件上运行，可能需要将权重从 bfloat16 转换为 float32。

精度要求：尽管 bfloat16 在训练时可以提供足够的精度，但在某些应用场景中，可能需要更高的精度来确保推理结果的准确性。在这种情况下，使用 float32 可以提供更高的数值精度。

软件支持：某些深度学习框架或库可能在处理 bfloat16 数据类型时存在限制或不足，这可能需要开发者在部署模型时将权重转换为 float32。

性能考虑：在某些情况下，使用 float32 进行推理可能比使用 bfloat16 更高效，尤其是在硬件对 float32 有优化的情况下。

通用性：float32 是深度学习中最常用的数据类型，许多库和工具都是围绕 float32 设计的。为了提高模型的通用性和兼容性，开发者可能会选择在推理时使用 float32。
"""
#如果使用2b的模型就使用2b的参数,否则使用7b的参数
model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
#导入分词器
model_config.tokenizer = f"{weights_dir}/tokenizer.model"
print(f"model_config:{model_config}")


print(f"model_config.get_dtype():{model_config.get_dtype()}") 
@contextlib.contextmanager#将函数转换成一个上下文管理器,临时更改数据的类型
def _set_default_tensor_type(dtype: torch.dtype):
    """Sets the default torch dtype to the given dtype."""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)
with _set_default_tensor_type(model_config.get_dtype()):
    #传入模型的结构,创建模型
    model = GemmaForCausalLM(model_config)
    print("This is the model structure. : \n", model)
    #根据路径去加载模型的权重
    model.load_weights(ckpt_path)
    print("Model weights loaded.")
    #模型移动到CPU并转成评估模式.
    model = model.to(device).eval()
print("Model Configuraiton Done")

model_config:GemmaConfig(vocab_size=256000, max_position_embeddings=8192, num_hidden_layers=18, num_attention_heads=8, num_key_value_heads=1, hidden_size=2048, intermediate_size=16384, head_dim=256, rms_norm_eps=1e-06, dtype='bfloat16', quant=False, tokenizer='/kaggle/input/gemma/pytorch/2b-it/2/tokenizer.model')
model_config.get_dtype():torch.bfloat16
This is the model structure. : 
 GemmaForCausalLM(
  (embedder): Embedding()
  (model): GemmaModel(
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (qkv_proj): Linear()
          (o_proj): Linear()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear()
          (up_proj): Linear()
          (down_proj): Linear()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (sampler): Sampler()
)


/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Model weights loaded.
Model Configuraiton Done


In [7]:
#查看相关的参数
help(GemmaForCausalLM.generate)

Help on function generate in module gemma.model:

generate(self, prompts: Union[str, Sequence[str]], device: Any, output_len: int = 100, temperature: Optional[float] = 0.95, top_p: float = 1.0, top_k: int = 100) -> Union[str, Sequence[str]]
    Generates responses for given prompts using Gemma model.



#### 为了考虑程序的运行时间,这里对2条测试数据共用一个预测结果。由于字数太少会扣分,这里对生成文本进行了微调。

In [8]:
import re#正则表达式库
#用户聊天模板和模型聊天模板
USER_CHAT_TEMPLATE = "<start_of_turn> user \n{prompt} <end_of_turn> \n"
MODEL_CHAT_TEMPLATE = "<start_of_turn> model\n{prompt} <end_of_turn> \n"
margin=2
for i in range(0,len(Input),margin):
    prompt = (
        #用户说中国有哪些可以旅游的地方
        USER_CHAT_TEMPLATE.format(
            prompt=f"你是一个厉害的小说家,请完成下面小说的创作,字数不少于1000字:{Input[i]},深呼吸,仔细思考,写出一篇精彩的小说。"
        )
        #模型开始说
        + "<start_of_turn>model\n"
    )
    output = model.generate(
        #用户说的提示词
        USER_CHAT_TEMPLATE.format(prompt=prompt),
        output_len=2048,
        device=device)
    for j in range(i,i+margin):
        Output[j]=output
    #字数太少会扣分
    if len(output)<=800:
        output="这是一个非常有趣的故事,且听我细细道来。"+output
    print(f"{i} is ok,len(output):{len(output)}")
print(f"Output[0]:{Output[0]}")

0 is ok,len(output):796
2 is ok,len(output):766
4 is ok,len(output):809
6 is ok,len(output):793
8 is ok,len(output):740
10 is ok,len(output):650
12 is ok,len(output):2854
14 is ok,len(output):665
16 is ok,len(output):823
18 is ok,len(output):800
20 is ok,len(output):820
22 is ok,len(output):829
24 is ok,len(output):931
26 is ok,len(output):686
28 is ok,len(output):679
30 is ok,len(output):777
32 is ok,len(output):817
34 is ok,len(output):1249
36 is ok,len(output):858
38 is ok,len(output):728
40 is ok,len(output):831
42 is ok,len(output):887
44 is ok,len(output):565
46 is ok,len(output):762
48 is ok,len(output):2929
50 is ok,len(output):694
52 is ok,len(output):2977
54 is ok,len(output):803
56 is ok,len(output):759
58 is ok,len(output):814
60 is ok,len(output):920
62 is ok,len(output):711
64 is ok,len(output):820
66 is ok,len(output):838
68 is ok,len(output):849
70 is ok,len(output):738
72 is ok,len(output):969
74 is ok,len(output):823
76 is ok,len(output):3577
78 is ok,len(output):821


#### 保存提交文件,这里需要命名为submit.json.

In [9]:
# 创建一个列表，包含所有行的字典
rows = []
for i in range(len(Output)):
    row = {"instruction": Instruction[i], "input": Input[i], "output": Output[i]}
    rows.append(row)
with open('submit.json', 'w') as f:
    for item in rows:
        # 将字典转换为JSON格式的字符串，并写入文件，每个对象占一行
        f.write(json.dumps(item) + '\n')